In [6]:
import pandas as pd
import pickle
import seaborn as sns
import numpy as np
path = '../../../../../../Desktop/Orthografietrainer2020/pickle/'

# Import table and keep relevant columns

In [7]:
infile = open(path+'sitzungssummary.pkl','rb')
session = pickle.load(infile)
infile.close()
session = session [['ID','UserID','UserAttribut','Art','HA','beendet','Fehler']]

In [3]:
infile = open(path+'saetze.pkl','rb')
sentences = pickle.load(infile)
infile.close()
sentences = sentences[['satzID','Schwierigkeit']]

In [4]:
infile = open(path+'schueler2020only.pkl','rb')
students = pickle.load(infile)
infile.close()
students = students[['ID','Geschlecht','Klassenstufe','Anmeldeklassenstufe','Aufgaben','Altaufgaben','done']]

In [5]:
infile = open(path+'xmlsaetze.pkl','rb')
xmlsentences = pickle.load(infile)
infile.close()
xmlsentences = xmlsentences[['ID','UserID','UebungsID','Testposition','SatzID','Erstloesung','Schussel','Datum', 'Erfolg','Loesungsnr']]

infile = open(path+'xmlsaetze_archiv.pkl','rb')
xmlsentences_archive = pickle.load(infile)
infile.close()
xmlsentences_archive = xmlsentences_archive[['ID','UserID','UebungsID','Testposition','SatzID','Erstloesung','Schussel','Datum', 'Erfolg','Loesungsnr']]

xmlsentences = xmlsentences.append(xmlsentences_archive)


# Clean tables

In [6]:
"""
clean session df
"""
session = session.dropna()
session.rename(columns = {'ID':'UebungsID'}, inplace = True)

## Userattribut: 1=student; 0= guest/teacher
session['UserAttribut'] = session['UserAttribut'].replace(['Schüler'],1)
session['UserAttribut'] = session['UserAttribut'].replace(['Gast','Lehrer'],0)
session[['UserAttribut']] = session[['UserAttribut']].astype('int16')
session['HA'] = session['HA'].replace(['frHA'],'Self')

In [7]:
"""
clean students df
"""
students.rename(columns = {'ID':'UserID'}, inplace = True)
students['done'] = students['done'].astype('int16')
##clean gender
students['Geschlecht'] = students['Geschlecht'].replace(['m.'],'m')
students['Geschlecht'] = students['Geschlecht'].replace(['w.','we'],'w')
students['Geschlecht'] = students['Geschlecht'].replace(['d','wm'],np.nan)
##class level: only 5-12
options = ['5', '6','7','8','9','10','11','12'] 
students = students[students['Klassenstufe'].isin(options)] 
students['Klassenstufe'] = students['Klassenstufe'].astype('int')
##years registered: create new column years registered from class level and date of registration
students = students[students['Anmeldeklassenstufe'].isin(options)] 
students['Anmeldeklassenstufe'] = students['Anmeldeklassenstufe'].astype('int')
students['Jahredabei'] = students['Klassenstufe'] - students['Anmeldeklassenstufe']
##Count of pending tasks: creat new column count of pending tasks
students['Aufgaben']=students['Aufgaben'].str.split()
students['AnzahlAufgaben'] = students['Aufgaben'].str.len()
students['AnzahlAufgaben'] = students['AnzahlAufgaben'].replace([np.nan],0)
students['AnzahlAufgaben'] = students['AnzahlAufgaben'].astype('int')
##Drop columns 
students= students.drop(columns=['Anmeldeklassenstufe','Aufgaben','Altaufgaben'])

In [8]:
"""
Merge xmlsentences with sentences
"""
xmlsentences.rename(columns = {'SatzID':'satzID'}, inplace = True)
xmlsentences= pd.merge(xmlsentences, sentences, on='satzID', how='inner')
##create date columns
xmlsentences['Datum']= pd.to_datetime(xmlsentences['Datum'])
xmlsentences['Uhrzeit'] = pd.DatetimeIndex(xmlsentences['Datum']).hour
xmlsentences['Wochentag'] = pd.DatetimeIndex(xmlsentences['Datum']).dayofweek
xmlsentences['Kalenderwoche'] = pd.DatetimeIndex(xmlsentences['Datum']).strftime("%V")
xmlsentences['Monat'] = pd.DatetimeIndex(xmlsentences['Datum']).month
xmlsentences['Tag'] = pd.DatetimeIndex(xmlsentences['Datum']).day

"""
function that determines if task was processed during school lesson
"""
def f(row):
    if row['Uhrzeit'] > 14:
        val = 0
    elif row['Uhrzeit'] < 8:
        val = 0
    else:
        val = 1
    return val
xmlsentences['ist_Schulzeit'] = xmlsentences.apply(f, axis=1)
xmlsentences = xmlsentences.drop(columns='UserID')

In [15]:
"""
filter
"""
## only use normal exercise sets and data from march and april
xmlsentences = xmlsentences[(xmlsentences.Testposition == 'version') | (xmlsentences.Testposition == 'pruefung') | (xmlsentences.Testposition == 'training')]
xmlsentences = xmlsentences[(xmlsentences.Monat == 3) | (xmlsentences.Monat == 4)]#|(xmlsentences.Monat == 5) | (xmlsentences.Monat == 6)

In [10]:
"""
create variable multiple wrong
"""
xmlsentences['Loesungsnr']=xmlsentences['Loesungsnr'].str.split()
xmlsentences['MehrfachFalsch'] = xmlsentences['Loesungsnr'].str.len()
xmlsentences['MehrfachFalsch'] = xmlsentences['MehrfachFalsch'].replace([np.nan],0)
xmlsentences['MehrfachFalsch'] = xmlsentences['MehrfachFalsch'].astype('int')
xmlsentences['MehrfachFalsch'] = xmlsentences['MehrfachFalsch'] - 1
##save preprocessed dataset
xmlsentences.to_pickle('xmlsentences_preprocessed.pkl')

Calculate dropout from session data

In [3]:
# infile = open('xmlsentences_preprocessed.pkl','rb')
# xmlsentences = pickle.load(infile)
# infile.close()

#split data
xmlsentences['group'] = xmlsentences.UebungsID % 10
xmlsentences1 = xmlsentences[xmlsentences.group == 1]
xmlsentences2 = xmlsentences[xmlsentences.group == 2]
xmlsentences3 = xmlsentences[xmlsentences.group == 3]
xmlsentences4 = xmlsentences[xmlsentences.group == 4]
xmlsentences5 = xmlsentences[xmlsentences.group == 5]
xmlsentences6 = xmlsentences[xmlsentences.group == 6]
xmlsentences7 = xmlsentences[xmlsentences.group == 7]
xmlsentences8 = xmlsentences[xmlsentences.group == 8]
xmlsentences9 = xmlsentences[xmlsentences.group == 9]
alle = [xmlsentences1, xmlsentences2, xmlsentences3, xmlsentences4, xmlsentences5, xmlsentences6, xmlsentences7, xmlsentences8, xmlsentences9]

In [ ]:
"""
function to find ordernumber and break
ordernumber == number of appearance of each sentence in a session, e.g. first sentence in a session = ordernumber 1
break == luecke == 1 if the session was paused more than 45 minutes; 0 otherwise
"""
cnt = 0

for x in alle:
    cnt = cnt +1
    num = str(cnt)
    
    session = x.UebungsID.unique()

    ## ordernumbers df: a ordernumber is calculated for each sentence in one session
    ordernumbers =pd.DataFrame(columns=['OrderNumber', 'ID'])
    for i in session:
        subset = x.loc[x['UebungsID'] == i]
        subset = subset.sort_values(by='Datum')
        subset = subset.reset_index(drop=True)
        subset.reset_index(inplace=True)
        subset = subset.rename(columns = {'index':'OrderNumber'})
        subset = subset[['OrderNumber','ID']]
        ordernumbers = ordernumbers.append(subset)

    ## merge ordernumber df to main data set
    x = pd.merge(x,ordernumbers,on='ID', how='left')

    x['OrderNumber'] = x['OrderNumber'] + 1
    x.to_pickle('xmlsentences'+num+'_ordernumber.pkl')
    x['Time'] = pd.DatetimeIndex(x['Datum']).strftime("%X")

    ## Create new variable luecke = is 1 if session is paused more than 45 minutes; otherwise 0 
    df_gr = x.groupby('UebungsID').agg({'Monat': ['min', 'max'], 'Tag': ['min', 'max'], 'Time': ['min', 'max']})
    df_gr = df_gr.reset_index()

    def f(row):
        if row.Monat['min'] != row.Monat['max']:
            val = 1
        elif  row.Tag['min'] != row.Tag['max']:
            val = 1
        elif  row.Time['min'] != row.Time['max']:
            if ((pd.to_datetime(row.Time['min']) + pd.to_timedelta(45, unit='m')) < pd.to_datetime(row.Time['max'])):
                val = 1
            else:
                val = 0
        else:
            val = 0
        return val

    df_gr['luecke'] = df_gr.apply(f, axis=1)
    df_gr.to_pickle('xmlsentences'+num+'_luecke.pkl')

In [38]:
"""
From Break Variable and Ordernumber information, the session nr is determined
session number = number of the session, when the user drops out and continues later
run code for all datasets (1-10)
"""

## load and prepare break information (SessionID and Break)
infile = open('files/xmlsentences1_luecke.pkl','rb')
xmlsentences1_luecke = pickle.load(infile)
infile.close()
xmlsentences1_luecke.columns = xmlsentences1_luecke.columns.droplevel(1)
xmlsentences1_luecke = xmlsentences1_luecke[['UebungsID','luecke']]

## load and prepare ordernumber of the sentence
infile = open('files/xmlsentences1_ordernumber.pkl','rb')
xmlsentences1_ordernumber = pickle.load(infile)
infile.close()

## merge dataframe
xmlsentences1_merged = pd.merge(xmlsentences1_ordernumber, xmlsentences1_luecke, on='UebungsID')

## split by break
xmlsentences1_luecke0 = xmlsentences1_merged[xmlsentences1_merged.luecke==0]
xmlsentences1_luecke1 = xmlsentences1_merged[xmlsentences1_merged.luecke==1]

## add variable session number = 1 when there is no break 
xmlsentences1_luecke0['sessionNr'] = 1

## calculate if a break is between two sentences from a session
## return 1 if there was a break, 0 if there was no break 
def check_if_break(aktuelltag, aktuellmonat, aktuelluhrzeit, vergleichtag, vergleichmonat, vergleichuhrzeit):
    if aktuellmonat.values[0] != vergleichmonat.values[0]:
        val = 1
    elif  aktuelltag.values[0] != vergleichtag.values[0]:
        val = 1
    elif  aktuelluhrzeit.values[0] != vergleichuhrzeit.values[0]:
        if (vergleichuhrzeit.values[0]+3) < aktuelluhrzeit.values[0]:
            val = 1
        else:
            val= 0
    else:
        val = 0
    return val

## prepare sessions where there is a dropout
errorlist = []
xml_with_breaks = xmlsentences1_luecke1
xml_with_breaks = xml_with_breaks.astype({'UebungsID': str})
xml_with_breaks['sessionNr'] = 0
session = xml_with_breaks.UebungsID.unique()

for x in session:
    try:
        ## get count of sentences per session
        anzahl = len(xml_with_breaks.loc[xml_with_breaks['UebungsID'] == x])

        ## get information from first sentence -> is definetly in session nr 1
        nr1 = xml_with_breaks.loc[(xml_with_breaks['UebungsID'] == x) & (xml_with_breaks.OrderNumber == 1)] 
        id1 = nr1.ID
        xml_with_breaks.loc[(xml_with_breaks.ID.values == id1.values), 'sessionNr'] = 1
        vergleichmonat = xml_with_breaks['Monat'].loc[(xml_with_breaks['UebungsID'] == x) & (xml_with_breaks.OrderNumber == 1)] 
        vergleichtag = xml_with_breaks['Tag'].loc[(xml_with_breaks['UebungsID'] == x) & (xml_with_breaks.OrderNumber == 1)]    
        vergleichuhrzeit = xml_with_breaks['Uhrzeit'].loc[(xml_with_breaks['UebungsID'] == x) & (xml_with_breaks.OrderNumber == 1)] 

        session = 1

        ## for each sentence in session
        for y in range(anzahl-1):
            ## get current sentence
            a = y+2
            nr = xml_with_breaks.loc[(xml_with_breaks['UebungsID'] == x) & (xml_with_breaks.OrderNumber == a)]
            this_id = nr.ID
            ## get date from current sentence
            aktuelltag = nr['Tag']
            aktuellmonat = nr['Monat']
            aktuelluhrzeit = nr['Uhrzeit']

            ## check if there is a break
            is_break = check_if_break(aktuelltag, aktuellmonat, aktuelluhrzeit, vergleichtag, vergleichmonat, vergleichuhrzeit)

            ## save session nr 
            if is_break == 1:
                xml_with_breaks.loc[(xml_with_breaks.ID.values == this_id.values), 'sessionNr'] = session+1
                session = session+1
            else:
                xml_with_breaks.loc[(xml_with_breaks.ID.values == this_id.values), 'sessionNr'] = session

            ## update date information to compare to the next sentence
            vergleichmonat = aktuellmonat
            vergleichtag = aktuelltag
            vergleichuhrzeit = aktuelluhrzeit

    except:
        print(x)

# save dataframe with new variable
xml_with_breaks.to_pickle('xmlsentences1_sessionnr1.pkl')
xmlsentences1_luecke0.to_pickle('xmlsatze1_sessionnr0.pkl')

In [21]:
"""
Finish dataset: new UebungsID, recalculate Ordernumber
run code for all datasets
"""

## load and append datasets
infile = open('xmlsentences1_sessionnr1.pkl','rb')
xmlsentences1_sessionnr1 = pickle.load(infile)
infile.close()
infile = open('xmlsentences1_sessionnr0.pkl','rb')
xmlsentences1_sessionnr0 = pickle.load(infile)
infile.close()
new_session_nr = xmlsentences1_sessionnr1.append(xmlsentences1_sessionnr0)

## Concatenate SessionNr to UebungsID 
new_session_nr['sessionNr'] = new_session_nr['sessionNr'].astype(str)
new_session_nr["UebungsID"] = new_session_nr["UebungsID"].map(str) + "_" + new_session_nr["sessionNr"]

## Recalculate ordernumber -> if there was a dropout, the second session begins with ordernumber 1
min_ordernr = new_session_nr.groupby('UebungsID')['OrderNumber'].min()
new_order = pd.merge(new_session_nr, min_ordernr, on='UebungsID')
new_order['OrderNumber_y'] = new_order['OrderNumber_y'] - 1
new_order['OrderNumber'] = new_order['OrderNumber_x'] - new_order['OrderNumber_y']
new_order = new_order.drop(columns=['OrderNumber_x', 'OrderNumber_y'])

## Create new column: dropped out before 
## if it is a second, third, fourth... session, this column indicates if the exercise was processed in more than one session
def f(row):
    if row['sessionNr'] != '1':
        val = 1
    else:
        val = 0
    return val

new_order['vorher_abgebrochen'] = new_order.apply(f, axis=1)
new_order.to_pickle('vacay_preprocessed1.pkl')